In [6]:
import heapq as hq

class Graph:

    def __init__(self, vertices):
        self.V = vertices
        self.graph = {}
                        
    # this method find the least common ancestor in the dumbest way
    # however by using skew-binary it will be better, but I did not understand that method to implement it
    def find_lca(self, x, parents_x, y, parents_y):
        # if x is root or the parent of y, then lca is x
        if parents_x == [] or x in parents_y:
            return x, []
        elif parents_y == [] or y in parents_x:
            return y, []
        
        # x = [3, 4, 5, 6]
        # y = [3, 4, 9, 8, 2, 15] -> y = [3, 4, 9, 8]
        # we can trim the longer parents because the lca is definitely not in there
        elif len(parents_x) < len(parents_y):
            p_y = parents_y[:len(parents_x)]
            p_x = parents_x
        else:
            p_y = parents_y
            p_x = parents_x[:len(parents_y)]
        # now iterate over the parents to find lca
        for i in range(len(p_y) -1, -1, -1):
            if p_y[i] == p_x[i]:
                return p_x[i], p_x[0: i-1]
        
        
    # the method for finding the composite distance of a node
    # start: the node we want to find its composite distance
    # graph: the graph that we need to find the composite distance in it
    def optimumCompositeDistance(self, start, graph):
        composite_distance = float('inf')
        distances = {}
        heap = [(0, start, [])]
        c = -1
        parents_c = [start]

        while heap:
            # pop the next node with the minimum distance to it  
            dist, node, parents = hq.heappop(heap)
            
            if node in distances or composite_distance <= dist:
                continue  # Already encountered before or we cannot find the minimum cycle after this node
                
            # We know that this is the first time we encounter node.
            #   As we pull nodes in order of increasing distance, this 
            #   must be the node's shortest distance from the start node.
            distances[node] = [dist, parents]
            
            # we process the neighbors of the node
            for neighbor, weight in graph[node]:
                # a cycle detected
                if neighbor in distances and neighbor != parents[-1]:
                    
                    lca, parents_lca = self.find_lca(neighbor, distances[neighbor][1], node, parents)
                    # calculate the composite distance
                    temp_cycle = dist+ weight+distances[neighbor][0] - distances[lca][0]
                    if temp_cycle < composite_distance:
                        composite_distance = temp_cycle
                        c = lca
                        parents_c = distances[lca][1]
                # # we have not encountered it before, so we push it to the heap to process it later
                elif neighbor not in distances:
                    # print(neighbor, distances)
                    temp = parents + [node]
                    hq.heappush(heap, (dist + weight, neighbor, temp))
        
        return composite_distance, c, parents_c
    
    
    # this method find the girth of the graph
    def optimumGirth(self):
        least_cycle = float('inf')
        # copy the graph without reference because we want to delete some part of it later
        temp_graph = {}
        for i in self.graph:
            temp_graph[i] = self.graph[i].copy()
        
        # process node by node to find the least cycle
        for node in self.graph.keys():
            # in the following searches we might delete some of nodes, so their keys will not be available anymore
            if node in temp_graph:
                # find the composite distance of a node, its first encountered node in the cycle, and the nodes between the cycle and the node(to delete them)
                composite_distance, c, parents_c = self.optimumCompositeDistance(node, temp_graph)
                # print(node, c)
                # if node is not a node in the cyle of composite distance, then we should delete the node and the nodes between that and the cycles - theorem 2
                if node != c:
                    # theorem 2
                    # nodes between the node and the cycle should be deleted
                    for i in parents_c:
                        # edges adjacent to those nodes should be deleted as well. 
                        for j, weight in temp_graph[i]:
                            for k in range(0, len(temp_graph[j])):
                                if temp_graph[j][k][0] == i:
                                    del temp_graph[j][k]
                                    # we assume there is one edge between any two nodes, so when we find that we can finish the loop
                                    break

                        del temp_graph[i]
                # print(temp_graph, '\n\n')
                if composite_distance < least_cycle:
                    least_cycle = composite_distance
        return least_cycle
        
        
    

g = Graph(8)

g.graph = {
    "a": [("b", 8), ("c", 3), ("d", 6)],
    "b": [("a", 8), ("e", 5)],
    "c": [("a", 3), ("d", 2)],
    "d": [("a", 6), ("c", 2), ("e", 15), ("g", 3)],
    "e": [("b", 5), ("d", 15), ("f", 5)],
    "f": [("e", 5), ("g", 30), ("h", 6)],
    "g": [("d", 3), ("f", 30), ("h", 4)],
    "h": [("g", 4), ("f", 6)]
}


# print(g.optimumCompositeDistance('b', g.graph)[0])
print(g.optimumGirth())


11
